#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [4]:
!unzip -j /Users/umesh/Desktop/"Week2 ML assignments"/utils.zip -d.

Archive:  /Users/umesh/Desktop/Week2 ML assignments/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


# Dataset Loading and Pre-Processing

In [6]:
import pandas as pd
data = pd.read_csv("weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [8]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
2246,19.0,71,3.5,35.0,overcast,1010.93,3,Spring,6.0,inland,Cloudy
606,29.0,46,6.0,9.0,clear,1014.94,5,Spring,6.5,inland,Sunny
11564,-9.0,81,10.5,91.0,overcast,991.28,0,Winter,2.0,mountain,Snowy
6201,-5.0,81,7.0,50.0,overcast,987.60,0,Winter,3.5,mountain,Snowy
4649,40.0,38,3.5,4.0,clear,1018.19,9,Autumn,8.5,inland,Sunny


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [9]:
# START CODE HERE
object_columns = data.select_dtypes(include='object').columns.tolist()
non_object_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [10]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [26]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

column_transformer = ColumnTransformer(
    [
        ('cat', OrdinalEncoder(), object_columns),
        ('num', StandardScaler(), non_object_columns)
    ]
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [27]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [30]:
X = data_scaled_labeled.drop(columns=['Weather Type'])
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [31]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [32]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,0.0,1.0,0.0,2.408445,0.757134,-0.482337,1.138057,0.553309,2.332257,-0.285616
12278,2.0,3.0,2.0,-1.675379,1.202822,1.471794,0.230256,-0.355875,-1.038715,-1.027154
2859,2.0,0.0,0.0,-0.179894,-0.431369,1.399419,0.918933,0.023982,-0.260799,-1.323769
5495,2.0,2.0,2.0,-0.064857,1.202822,1.109918,-0.020172,0.163774,-0.779410,-1.027154
8469,2.0,2.0,2.0,-0.064857,0.014320,0.530916,-0.333207,-0.062850,-0.001493,0.604230


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [33]:
# START CODE HERE

from sklearn import svm

model = svm.SVC()
model.fit(X_train, y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 90.30303030303031


**TEST** (DON'T EDIT THIS CELL)

In [34]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [35]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,3.0,2.0,2.0,5.054302,1.896115,0.748042,0.918933,0.246574,1.813646,0.455922
2331,3.0,2.0,1.0,-0.352450,-0.629452,-0.916588,-1.209704,-0.090002,-0.520104,0.455922
217,3.0,2.0,1.0,-0.467487,1.252343,-0.048086,0.731112,0.207862,-1.038715,-0.730539
1760,0.0,1.0,1.0,0.797923,-1.273224,-1.278464,-1.178401,0.506533,0.776424,-0.137308
3963,3.0,2.0,1.0,-0.179894,0.460008,-0.699463,-0.427117,0.342815,-0.260799,0.604230


In [36]:
y = model.predict(input)
y

array([3., 0., 1., 3., 0.])

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
